In [1]:
import torch
import datasets
import json
from pathlib import Path


In [2]:
ds = datasets.load_dataset("atin5551/reddit-story-niche-classification-dataset")
data_out = Path("data").expanduser()
system_prompt_OLMo = "You are OLMo 2, a helpful and harmless AI Assistant built by the Allen Institute for AI."
system_prompt_llama = "You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.  If you don't know the answer to a question, please don't share false information."

In [3]:
print(ds["train"][0].keys())
print(ds["train"][0])
print(len(ds["train"]))
print(len(ds["test"]))

dict_keys(['title', 'selftext', 'subreddit', 'flair', 'score', 'num_comments', 'upvote_ratio', 'created_utc', 'id', 'url', 'niche', 'title_length', 'contains_question', 'contains_capslock', 'engagement_score', 'hour_of_posting', 'selftext_length', '__index_level_0__'])
{'title': 'Before we were officially exclusive, but AFTER we\'d had a couple of "magical" dates, my [M29] current girlfriend [F24] of 3.5 months slept with a friend of hers, and separately had an MFM threesome. How should I feel?', 'selftext': 'My girlfriend Alice and I had a whirlwind romance and immediately fell for one another. Honestly, it was such a breath of fresh air, I have been dating a "certain type" of girl for so long and she was so different and friendly - to meet someone I had such an instant connection with who also had all my hobbies and gave me all the best, healthy feedback was amazing. My previous relationships have been toxic or just dumb, like I was killing time with them. I was immediately in love, 

In [4]:


def format_sample(sample: dict[str, str]):
    selftext = sample["selftext"].split(".")
    if len(selftext) < 2:
        return None # cannot predict second sentence
    system_prompt_a = "System: " + system_prompt_OLMo
    task_input_a = f"{system_prompt_a} ---- User: {sample['title']} {selftext[0]}"
    answer_a = f"Assistant: <reddit>{sample['subreddit']}</reddit>"
 
    system_prompt_b = "System: " + system_prompt_llama
    task_input_b = f"{system_prompt_b} ---- User: {sample['title']} {selftext[0]}"
    answer_b = f"Assistant: <story> {selftext[1]} </story>"


    return {"task_input_a": task_input_a, "task_input_b": task_input_b, "task_answer_a": answer_a, "task_answer_b": answer_b}

print(format_sample(ds["train"][0]))

folder = data_out / "title_and_first_sen"
folder.mkdir(parents=True, exist_ok=True)
# save the first dataset line to a jsonl file
print(folder)
with open(folder / "data-train.jsonl", "w") as f:
    for sample in ds["train"]:
        data_line = format_sample(sample)
        if data_line is not None:
            f.write(json.dumps(data_line) + "\n")

with open(folder / "data-test.jsonl", "w") as f:
    for sample in ds["test"]:
        data_line = format_sample(sample)
        if data_line is not None:
            f.write(json.dumps(data_line) + "\n")




{'task_input_a': 'System: You are OLMo 2, a helpful and harmless AI Assistant built by the Allen Institute for AI. ---- User: Before we were officially exclusive, but AFTER we\'d had a couple of "magical" dates, my [M29] current girlfriend [F24] of 3.5 months slept with a friend of hers, and separately had an MFM threesome. How should I feel? My girlfriend Alice and I had a whirlwind romance and immediately fell for one another', 'task_input_b': 'System: You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.  If you don\'t know the answer to a question, please don\'t share false information. ---- User: Before we were offic